In [396]:
import pandas as pd

In [397]:
import glob

In [398]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [399]:
import json
f = open('data/entities.json')
data = json.load(f)

In [400]:
d={}
for ent in data:
    row=pd.DataFrame.from_dict(ent['attributes'], orient='index')
    d[ent['id']-1]=row.transpose().iloc[0]
data=pd.DataFrame.from_dict(d,orient='index')

In [401]:
data=data[data['primary_ext']=='Person']

In [402]:
data.shape

(284620, 13)

In [403]:
data['id'].nunique()

284620

In [404]:
data['name'].nunique()

273467

In [405]:
data_copy=data.copy()

In [406]:
data['blurb'] = data['name'] + ' is a ' + data['blurb'] + '.'

In [407]:
# data['summary']=
data['summary'].fillna('').str.split('.').apply(lambda x: '. '.join(x[:2]) + '.')

1005           Mr.  Questrom was the Chairman and CEO of J.
1006      Ms.  Wolf is the former Chairman and CEO of Le...
1007      Ms.  Alvarez is the former Administrator of th...
1008      Mr.  Penner has been a General Partner at Madr...
1009      Mr.  Breyer is a Managing Partner of Accel Par...
                                ...                        
427650                                                    .
427651                                                    .
427652                                                    .
427654                                                    .
427656                                                    .
Name: summary, Length: 284620, dtype: object

In [408]:
# Transform start date into sentence
data.loc[~data['start_date'].isnull(),'start_date_sentence']=data.loc[~data['start_date'].isnull(),'start_date'].apply(lambda x: f'This person was born in {x}.')

In [409]:
# Transform end date into sentence
data.loc[~data['end_date'].isnull(),'end_date_sentence']=data.loc[~data['end_date'].isnull(),'end_date'].apply(lambda x: f'This person died in {x}.')

In [410]:
# Transform types into sentence
data['types']=data['types'].apply(lambda x:' '.join(x)).str.replace('Person','')
data['types']=data['types'].str.replace(' ',', ').apply(lambda x: 'This person is associated with: '+x[1:]+'.')
# Fix typos
data['types']=data['types'].str.replace(', ,',',').str.replace(', \.','.').str.replace('Media, ality','Media Personality')
# Remove sentences without context 
data.loc[data['types']=='This person is associated with: .','types']=''

/tmp/ipykernel_1762/3035051205.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  data['types']=data['types'].str.replace(', ,',',').str.replace(', \.','.').str.replace('Media, ality','Media Personality')


In [411]:
# Create context 
data['context'] = data['blurb'].fillna('') + ' ' + data['summary'].fillna('')
data['context']=data['context'].str.replace('\r','').str.replace('\n', '')

In [412]:
data.drop(['blurb','summary', 'updated_at', 'parent_id'],1,inplace=True)

/tmp/ipykernel_1762/2472487714.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data.drop(['blurb','summary', 'updated_at', 'parent_id'],1,inplace=True)


In [413]:
data.shape

(284620, 12)

In [414]:
data.head(3)

,id,name,website,primary_ext,start_date,end_date,aliases,types,extensions,start_date_sentence,end_date_sentence,context
1005,1006,Allen I Questrom,None,Person,1940-04-13,None,"[Allen I Questrom, Mr Allen Irving Questrom]",This person is associated with: Business.,"{'Person': {'name_last': 'Questrom', 'name_fir...",This person was born in 1940-04-13.,NaN,Allen I Questrom is a former chairman & CEO of...
1006,1007,Linda S Wolf,None,Person,None,None,"[Linda S Wolf, Ms Linda S. Wolf]",This person is associated with: Business.,"{'Person': {'name_last': 'Wolf', 'name_first':...",NaN,NaN,Ms. Wolf is the former Chairman and CEO of Le...
1007,1008,Aida M Alvarez,None,Person,None,None,"[Aida M Alvarez, Ms Aida M. Alvarez]",This person is associated with: Business.,"{'Person': {'name_last': 'Alvarez', 'name_firs...",NaN,NaN,Aida M Alvarez is a former administrator of US...


In [415]:
data['primary_ext'].unique()

array(['Person'], dtype=object)

In [416]:
def combine_dictionaries(list_of_dicts):
    result = {}
    for d in list_of_dicts:
        result.update(d)
    return result

In [417]:
data['extensions']=data['extensions'].apply(lambda r: [r[key] for key in r.keys()])

In [418]:
unique_extension_keys_extension_keys=[]
df=data['extensions'].apply(lambda r: list(combine_dictionaries(r).keys()))
for row in df:
    unique_extension_keys_extension_keys.extend(row)
del(df)

In [419]:
# manual selection - most other fields are useless, sparse or null
extensions_fields=['birthplace']
valid_indices=data[data['extensions'].apply(lambda x: len(x))>0].index.values
for field in extensions_fields:
    data.loc[valid_indices,field]=data.loc[valid_indices,'extensions'].apply(lambda x: [x[0][field] if field in x[0] else None][0])

In [420]:
data.drop(['extensions'],1,inplace=True)

/tmp/ipykernel_1762/1391979894.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data.drop(['extensions'],1,inplace=True)


In [421]:
# Transform birth place into sentence
data.loc[~data['birthplace'].isnull(),'birthplace']=data.loc[~data['birthplace'].isnull(),'birthplace'].apply(lambda x: f'This person was born in {x}.')

In [422]:
extra_context_cols=['start_date_sentence','end_date_sentence','types','birthplace']
for col in extra_context_cols:
    data['context'] = data['context'] + ' ' + data[col].fillna('')

In [423]:
data.to_csv('data/kb_datasets/lilsis_entities.csv')

In [424]:
# stats on number of entities sharing first and/or last name

In [425]:
data['first_name']=data['name'].apply(lambda r: r.split(' ')[0])

In [426]:
data['last_name']=data['name'].apply(lambda r: r.split(' ')[-1])

In [144]:
first_name_counts=data.groupby(['first_name'])['first_name'].count()

In [148]:
first_name_counts=first_name_counts.to_frame(name='counts').reset_index()

In [159]:
first_name_counts.sort_values(by='counts', ascending=False).head(10)

,first_name,counts
27442,John,9147
14172,David,6898
35429,Michael,6374
45063,Robert,6154
26634,James,5483
56678,William,4716
44775,Richard,3827
52563,Thomas,3416
33964,Mark,3298
40847,Paul,2787


In [160]:
last_name_counts=data.groupby(['last_name'])['last_name'].count()

In [161]:
last_name_counts=last_name_counts.to_frame(name='counts').reset_index()

In [162]:
last_name_counts.sort_values(by='counts', ascending=False).head(10)

,last_name,counts
52982,LLC,5914
21311,Congress,5403
47320,Jr,4419
45360,Inc.,4207
33591,Foundation,4128
45355,Inc,3446
21110,Committee,2362
39099,Group,2204
34447,Fund,2130
21852,Corporation,2063


In [169]:
same_name_counts=data.groupby(['first_name','last_name'])['first_name'].count()

In [170]:
same_name_counts=same_name_counts.to_frame(name='counts').reset_index()

In [178]:
same_name_counts.sort_values(by='counts', ascending=False).head(100)

,first_name,last_name,counts
165326,John,Jr,342
340410,William,Jr,303
316713,The,Foundation,290
276773,Robert,Jr,267
316743,The,Group,230
147835,James,Jr,223
53504,Charles,Jr,167
165238,John,III,156
318848,Thomas,Jr,148
340348,William,III,142


In [179]:
data[(data['first_name']=='John')&(data['last_name']=='Williams')]

,id,name,website,primary_ext,start_date,end_date,aliases,types,extensions,context,first_name,last_name
2539,2540,John Y Williams,None,Person,1943-00-00,None,[John Y Williams],"[Person, Business Person]",Williams John Y 2 1196488,John Y. Williams has served as a Director sin...,John,Williams
5502,5503,John H Williams,None,Person,1934-00-00,None,[John H Williams],"[Person, Business Person]",Williams John H 2 1014058,John H. Williams was a Senior Vice President ...,John,Williams
14066,14067,John Patrick Williams,None,Person,1937-10-30,None,[John Patrick Williams],"[Person, Political Candidate, Elected Represen...",Williams John Patrick 2 12886 False W000520 41...,US Representative from Montana A Representativ...,John,Williams
16000,16001,John Williams,None,Person,None,None,[John Williams],"[Person, Lobbyist]",Williams John,Lobbyist,John,Williams
33937,33938,John R Williams,None,Person,1909-10-29,None,[John R Williams],"[Person, Political Candidate, Elected Represen...",Williams John R Jack California 12901 False True,"Former Governor of Arizona JOHN ""JACK"" R. WILL...",John,Williams
54970,54971,John Graham Williams,None,Person,None,None,[John Graham Williams],[Person],Williams John Graham 52788,Prospective Parliamentary Candidate for South ...,John,Williams
55086,55087,John Williams,None,Person,None,None,[John Williams],[Person],Williams John 52780,Prospective Parliamentary Candidate for Hackne...,John,Williams
62206,62207,John A Williams,None,Person,None,None,"[Dr. John A. Williams, John A Williams]","[Person, Business Person]",Williams John A Mr 2 1350063,,John,Williams
66470,66471,John D Williams,None,Person,None,None,[John D Williams],[Person],Williams John D 2,"President and Chief Executive Officer, Domtar ...",John,Williams
85863,85864,John H Williams,None,Person,None,None,[John H Williams],"[Person, Business Person]","Williams John H Havanna, Cuba 2",,John,Williams


In [37]:
i=0
for gu_ner_file in glob.glob('data/extracted_named_entities_output/sampled_*'):
    gu_ents = pd.read_csv(gu_ner_file,index_col=0)
    if i==0:
        gu_ents_data=gu_ents
    else:
        gu_ents_data=pd.concat([gu_ents_data,gu_ents],ignore_index=True)
    i+=1

In [38]:
gu_ents_data.head()

,text,label,start,end,start_char,end_char,doc_index
0,Zachary Rolfe,PERSON,5,7,22,35,0
1,the Northern Territory,GPE,14,17,73,95,0
2,A week,DATE,22,24,122,128,0
3,John Burns,PERSON,66,68,358,368,0
4,Rolfe,PERSON,80,81,438,443,0


In [39]:
gu_ents_data.shape

(11868130, 7)

In [40]:
gu_ents_data.drop_duplicates(inplace=True)

In [42]:
sis_ents=set(data['name'])
unique_gu_ents=set()
for gu_ner_file in glob.glob('data/extracted_named_entities_output/sampled_*'):
    gu_ents = pd.read_csv(gu_ner_file,index_col=0)
    unique_gu_ents = unique_gu_ents | set(gu_ents['text'])

In [43]:
common_ents=[ent for ent in unique_gu_ents if ent in sis_ents]

In [44]:
common_gu_ents=''
first_cycle=1
for gu_ner_file in glob.glob('data/extracted_named_entities_output/sampled_*'):
    year=gu_ner_file.split('_')[-2]
    gu_ents = pd.read_csv(gu_ner_file,index_col=0)
    gu_ents['article_year']=year
    gu_ents=gu_ents[gu_ents['text'].isin(common_ents)]
    if first_cycle:
        common_gu_ents=gu_ents
        first_cycle=0
    else:
        common_gu_ents=pd.concat([common_gu_ents, gu_ents],ignore_index=True)

In [45]:
common_gu_ents.head()

,text,label,start,end,start_char,end_char,doc_index,article_year
0,John Burns,PERSON,66,68,358,368,0,2022
1,Independent,ORG,140,141,691,702,1,2022
2,Independent,ORG,530,531,2697,2708,1,2022
3,Sonia Sotomayor,PERSON,7,9,42,57,4,2022
4,Samuel Alito,PERSON,151,153,830,842,4,2022


In [46]:
common_ents_data=common_gu_ents.merge(data_original, left_on='text', right_on='name')

In [47]:
common_ents_data=common_ents_data.drop(['text','website','parent_id','primary_ext'],1)

/tmp/ipykernel_1724/3769931002.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  common_ents_data=common_ents_data.drop(['text','website','parent_id','primary_ext'],1)


In [48]:
common_ents_data.head()

,label,start,end,start_char,end_char,doc_index,article_year,id,name,start_date,end_date,aliases,types,extensions,context
0,PERSON,66,68,358,368,0,2022,41210,John Burns,None,None,[John Burns],"[Person, Business Person]","{'Person': {'name_last': 'Burns', 'name_first'...","General Partner, Highland Consumer Fund Prior ..."
1,PERSON,192,194,1012,1022,2992,2022,41210,John Burns,None,None,[John Burns],"[Person, Business Person]","{'Person': {'name_last': 'Burns', 'name_first'...","General Partner, Highland Consumer Fund Prior ..."
2,PERSON,881,883,4412,4422,9130,2022,41210,John Burns,None,None,[John Burns],"[Person, Business Person]","{'Person': {'name_last': 'Burns', 'name_first'...","General Partner, Highland Consumer Fund Prior ..."
3,PERSON,47,49,263,273,9157,2022,41210,John Burns,None,None,[John Burns],"[Person, Business Person]","{'Person': {'name_last': 'Burns', 'name_first'...","General Partner, Highland Consumer Fund Prior ..."
4,PERSON,140,142,718,728,9974,2022,41210,John Burns,None,None,[John Burns],"[Person, Business Person]","{'Person': {'name_last': 'Burns', 'name_first'...","General Partner, Highland Consumer Fund Prior ..."


In [34]:
common_ents_data.shape

(840642, 15)

In [35]:
common_ents_data.head(2)

,label,start,end,start_char,end_char,doc_index,article_year,id,name,start_date,end_date,aliases,types,extensions,context
0,PERSON,66,68,358,368,0,2022,41210,John Burns,None,None,[John Burns],"[Person, Business Person]","{'Person': {'name_last': 'Burns', 'name_first'...","General Partner, Highland Consumer Fund Prior ..."
1,PERSON,192,194,1012,1022,2992,2022,41210,John Burns,None,None,[John Burns],"[Person, Business Person]","{'Person': {'name_last': 'Burns', 'name_first'...","General Partner, Highland Consumer Fund Prior ..."


In [36]:
common_ents_data.loc[common_ents_data['label']=='ORG','name'].nunique()

6592

In [37]:
common_ents_data.loc[common_ents_data['label']=='PERSON','name'].nunique()

26164

In [38]:
common_ents_data.loc[(common_ents_data['context']!=' ')&(common_ents_data['label']=='PERSON'), 'name'].nunique()

20862

In [39]:
common_ents_data.loc[(common_ents_data['context']!=' ')&(common_ents_data['label']=='ORG'), 'name'].nunique()

4460

In [40]:
gu_body_texts=''
first_cycle=1
for gu_file in glob.glob('data/sampled_*'):
    year=gu_file.split('_')[-1].split('.')[-2]
    gu_article=pd.read_csv(gu_file)
    gu_article=gu_article[['body_text']]
    gu_article['article_year']=int(year)
    gu_article['doc_index']=gu_article.index.values
    if first_cycle:
        gu_body_texts=gu_article
        first_cycle=0
    else:
        gu_body_texts=pd.concat([gu_body_texts, gu_article],ignore_index=True)

In [45]:
common_ents_data['article_year']=common_ents_data['article_year'].astype(int)

In [46]:
common_ents_data=common_ents_data.merge(gu_body_texts, how='left', on=['article_year','doc_index'])

In [59]:
common_ents_data[['body_text','name','label','article_year','doc_index','id','aliases','types','context']].sample(20)

,body_text,name,label,article_year,doc_index,id,aliases,types,context
593382,Planning permission deadlines are to be extend...,Richard Desmond,PERSON,2020,10023,81199,[Richard Desmond],"[Person, Business Person]",British publishing billionaire Tabloid king De...
557192,Boris Johnson’s overhauled cabinet has agreed ...,Ian Murray,PERSON,2020,12558,142626,[Ian Murray],[Person],"Vice President, Competitive Enterprise Institu..."
760899,We’ve launched a new blog at the link below – ...,Simon Smith,PERSON,2021,5544,310204,"[Mr Simon Smith, Simon Smith]",[Person],
582541,Florida’s ‘don’t say gay’ law officially goes ...,Ghislaine Maxwell,PERSON,2022,8691,176895,[Ghislaine Maxwell],[Person],Daughter of disgraced British newspaperman Rob...
687195,Environment minister Melissa Price requested a...,Trevor St Baker,PERSON,2019,16701,426478,[Trevor St Baker],"[Person, Business Person]",Major shareholder Tritium | Founder St Baker E...
574700,Outstanding lead actor in a comedy Nominees: A...,Maisie Williams,PERSON,2016,22731,247842,[Maisie Williams],"[Person, Media Personality]",Actress
677157,The capital's newly launched TV channel London...,Sarah Sands,PERSON,2014,5555,366269,[Sarah Sands],[Person],Spouse of Gregory Sands
767669,This blog is closing now but you can follow al...,Fred Hutchinson Cancer Research Center,ORG,2021,4953,39987,[Fred Hutchinson Cancer Research Center],"[Organization, Business]",
76125,The former Republican senator Tom Coburn has d...,Mike Pence,PERSON,2020,9557,394860,[Mike Pence],[Person],
373759,It said much about Boris Johnson’s eight-year ...,Sadiq Khan,PERSON,2016,22903,56708,[Sadiq Khan],[Person],Prospective Parliamentary Candidate for Tootin...


In [65]:
common_ents_data.loc[(common_ents_data['doc_index']==4953)&(common_ents_data['article_year']==2018),['body_text','name','label','article_year','doc_index','id','aliases','types','context']]

,body_text,name,label,article_year,doc_index,id,aliases,types,context
191560,Harvey Weinstein has apologised for using stat...,Meryl Streep,PERSON,2018,4953,72380,[Meryl Streep],"[Person, Media Personality]","Actress Generally considered, without hyperbol..."
191561,Harvey Weinstein has apologised for using stat...,Meryl Streep,PERSON,2018,4953,72380,[Meryl Streep],"[Person, Media Personality]","Actress Generally considered, without hyperbol..."
200963,Harvey Weinstein has apologised for using stat...,Gwyneth Paltrow,PERSON,2018,4953,49084,"[Gwyneth Martin, Gwyneth Paltrow]",[Person],Actress; founder of Goop
215663,Harvey Weinstein has apologised for using stat...,Harvey Weinstein,PERSON,2018,4953,46257,[Harvey Weinstein],"[Person, Business Person]",Film Producer; Co-founder of Miramax The movie...
215664,Harvey Weinstein has apologised for using stat...,Harvey Weinstein,PERSON,2018,4953,46257,[Harvey Weinstein],"[Person, Business Person]",Film Producer; Co-founder of Miramax The movie...
405659,Harvey Weinstein has apologised for using stat...,Jennifer Lawrence,PERSON,2018,4953,149040,[Jennifer Lawrence],"[Person, Media Personality]",Actress
405660,Harvey Weinstein has apologised for using stat...,Jennifer Lawrence,PERSON,2018,4953,412275,"[Jennifer Lawrence, Jennifer Lynn Lawrence]",[Person],"Dustin Stockton fiancee, America First rally c..."
405661,Harvey Weinstein has apologised for using stat...,Jennifer Lawrence,PERSON,2018,4953,149040,[Jennifer Lawrence],"[Person, Media Personality]",Actress
405662,Harvey Weinstein has apologised for using stat...,Jennifer Lawrence,PERSON,2018,4953,412275,"[Jennifer Lawrence, Jennifer Lynn Lawrence]",[Person],"Dustin Stockton fiancee, America First rally c..."
697672,Harvey Weinstein has apologised for using stat...,The Weinstein Company,ORG,2018,4953,273721,"[TWC, The Weinstein Company, The Weinstein Com...","[Organization, Business, Private Company]",Film production company The Weinstein Company...


In [68]:
common_ents_data[common_ents_data['article_year']==2022].to_csv('lilsis_gu_intersection_common_ents_data_2022.csv')

In [48]:
common_ents_data[common_ents_data['body_text'].isna()]

,label,start,end,start_char,end_char,doc_index,article_year,id,name,start_date,end_date,aliases,types,extensions,context,body_text


In [56]:
common_ents_data.loc[~(common_ents_data['context']==' '),'name'].nunique()

25123

In [49]:
common_ents_data.to_csv('lilsis_gu_intersection_common_ents_data.csv')

In [ ]:
intersection of open sanctions and little sis 